# Set env

In [9]:
import dotenv
dotenv.load_dotenv('../.env')
import os

# Species list for a location

In [ ]:
import requests
import pandas as pd

In [ ]:
headers = {
    "x-ebirdapitoken": os.getenv("x-ebirdapitoken")
}

In [ ]:
locId="L6183959"
# url= f"https://api.ebird.org/v2/ref/hotspot/info/{locId}"
url=f"https://api.ebird.org/v2/product/spplist/{locId}"
birdlistforloc=requests.get(url, headers=headers).json()


In [ ]:
url="https://api.ebird.org/v2/ref/taxonomy/ebird?fmt=json"
allBirds=requests.get(url, headers=headers).json()
allBirdsdf=pd.json_normalize(allBirds)

In [ ]:
allBirdsdf= allBirdsdf[allBirdsdf['category'] == "species"]

In [ ]:

new_df = pd.DataFrame({'speciesCode': birdlistforloc}).merge(allBirdsdf[["comName","speciesCode"]], on='speciesCode')

In [ ]:
new_df

# reading text from images

In [ ]:
import pytesseract
from PIL import Image



In [ ]:
# Path to the Tesseract executable (change it according to your installation)
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'

# Open the image file
image = Image.open('/Users/joy/akvo/garbage/ss/Deden.png')




English (eng)
Spanish (spa)
French (fra)
German (deu)
Italian (ita)
Portuguese (por)
Dutch (nld)
Russian (rus)
Chinese (chi_sim for Simplified Chinese and chi_tra for Traditional Chinese)
Japanese (jpn)
Korean (kor)
Arabic (ara)
Hindi (hin)
Bengali (ben)
Tamil (tam)

In [ ]:
# Convert the image to grayscale
image = image.convert('L')
languages = ['eng']
# Use Tesseract to extract text from the image
extracted_text = ""

# Iterate over each language and extract text
for lang in languages:
    text = pytesseract.image_to_string(image, lang=lang)
    extracted_text += text + "\n"

# Print the extracted text
print(extracted_text)

# Accuweather API

In [ ]:
import requests
import pandas as pd
import json
from urllib.parse import quote

In [ ]:
apiKey=os.getenv("accu_apiKey")

In [ ]:
latLong=quote('28.583343,77.174635')
locationKey= requests.get(f'http://dataservice.accuweather.com/locations/v1/cities/geoposition/search?apikey={apiKey}&q={latLong}').json()['Key']


In [ ]:
forecast=requests.get(f'http://dataservice.accuweather.com/forecasts/v1/daily/5day/{locationKey}?apikey={apiKey}&details=true&metric=true').json()
forecast

In [ ]:
dates = [user['Day']["Rain"]["Value"] for user in forecast["DailyForecasts"]]
dates

# Bulk download youtube video

In [ ]:
from googleapiclient.discovery import build
import json
import pandas as pd
import os
import subprocess

In [ ]:


# Set up the YouTube Data API client
api_key = os.getenv("youtube_apikey")  # Replace with your actual API key
youtube = build('youtube', 'v3', developerKey=api_key)

# Define the search query
search_query = "Pratham Alo ( প্রথম আলো ) by Sunil Gangopadhyay ; Part"
channel_id = "UCK-g7XJuUzonElW_pMJVAsw"  # Replace with the actual channel ID
output_directory = os.path.expanduser("~/Downloads/Series/PP")
# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)




In [ ]:
max_results = 100
next_page_token = None

# Create empty lists to store video links, titles, and channel IDs
video_links = []
video_titles = []
channel_ids = []

# Retrieve all available search results
while True:
    # Call the search.list method to retrieve the search results
    search_response = youtube.search().list(
        q=search_query,
        channelId=channel_id,
        part='id,snippet',
        maxResults=max_results,
        pageToken=next_page_token
    ).execute()

    # Iterate over the search results and add the video links, titles, and channel IDs to the lists
    for item in search_response['items']:
        if 'videoId' in item['id']:
            video_id = item['id']['videoId']
            video_link = f"https://www.youtube.com/watch?v={video_id}"
            video_title = item['snippet']['title']
            channel_id = item['snippet']['channelId']
            video_links.append(video_link)
            video_titles.append(video_title)
            channel_ids.append(channel_id)

    # Check if there are more pages of results
    if 'nextPageToken' in search_response:
        next_page_token = search_response['nextPageToken']
    else:
        break

# Create a DataFrame from the lists of video links, titles, and channel IDs
df_videos = pd.DataFrame({
    'Video Link': video_links,
    'Video Title': video_titles,
    'Channel ID': channel_ids
})

# Filter the DataFrame to include only videos from the specified channel ID
filtered_df = df_videos[df_videos['Channel ID'] == channel_id]




In [ ]:


# Filter the DataFrame to include only rows where the video title matches the criteria . In this case we check if a number above 47 is part of the title
filtered_df = df_videos[
    df_videos['Video Title'].str.contains("Pratham Alo") &
    df_videos['Video Title'].str.contains(r"\b(?:4[8-9]|[5-9]\d+)\b", regex=True)
]




In [ ]:


for index, row in filtered_df.iterrows():
    numbers = re.findall(r'\b(?:4[8-9]|[5-9]\d+)\b', row['Video Title'])  # Extract numbers above 48 using regex pattern

    # Check if numbers are found in the title
    if numbers:
        number = numbers[0]  # Get the first number
        filename = f"PA-{number}.mp3"
        command = [
            "yt-dlp",
            "-x",
            "--audio-format",
            "mp3",
            "--output",
            f"{output_directory}/{filename}",
            row['Video Link']
        ]
        subprocess.run(command) 
# TEST WITH ONE RECORD
# first_row = filtered_df.iloc[0]

# # Extract the number from the video title
# numbers = re.findall(r'\b(?:4[8-9]|[5-9]\d+)\b', first_row['Video Title'])
# if numbers:
#     number = numbers[0]
#     filename = f"PA-{number}.mp3"
#     command = [
#         "yt-dlp",
#         "-x",
#         "--audio-format",
#         "mp3",
#         "--output",
#         f"{output_directory}/{filename}",
#         first_row['Video Link']
#     ]
#     subprocess.run(command)

# Text to speech

In [ ]:
from gtts import gTTS
import os

In [ ]:
weather_advisory = """
Weather Advisory - Next 5 Days:

Day 1:
- Min Temperature: 15°C
- Max Temperature: 27°C
- Rainfall Forecast: Light showers expected.

Day 2:
- Min Temperature: 14°C
- Max Temperature: 26°C
- Rainfall Forecast: Mostly clear skies, no rainfall expected.

Day 3:
- Min Temperature: 16°C
- Max Temperature: 28°C
- Rainfall Forecast: Heavy rain expected in the evening.

Day 4:
- Min Temperature: 17°C
- Max Temperature: 29°C
- Rainfall Forecast: Partly cloudy, chance of scattered showers.

Day 5:
- Min Temperature: 18°C
- Max Temperature: 30°C
- Rainfall Forecast: Thunderstorms likely in the afternoon.

Please stay prepared and take necessary precautions for any weather changes.
"""

In [ ]:
def text_to_speech(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("output.mp3")  # Save the speech to a file in the current directory



In [ ]:
if __name__ == "__main__":
    text_to_speech(weather_advisory)

# Jugalbandi

In [ ]:
import requests
import json
from urllib.parse import urlencode, quote

In [ ]:
params = {
    'uuid_number': '50296d62-2df8-11ee-ba3a-515343c435e8',
    'query_string': 'Which province grows the most rice'
    
}
params=urlencode(params, quote_via=quote)
url=f'https://api.jugalbandi.ai/query-with-gptindex?{params}'
url

In [ ]:
content= requests.get(url).json()
content

# Exploring Kenya data archieve 

In [ ]:
import pandas as pd
import pyreadstat

In [ ]:
spss_file_path="/Users/joy/Downloads/SPSS/KEPR8ASV/KEPR8AFL.SAV"
# Read the SPSS file into a pandas DataFrame and get the metadata
df, meta = pyreadstat.read_sav(spss_file_path)

# Access the variable labels (column descriptions) from the metadata
variable_labels = meta.column_labels

# Print the variable labels for each column


In [ ]:

df.columns = [variable_labels.get(col, col) for col in df.columns]

# Now, the DataFrame has column names as the variable labels (descriptions)
print(df.head())

# Bulk rename files

In [ ]:
import os
import re
folder_path="/Users/joy/Downloads/series/PP"
# search_pattern = r'পর্ব\s*-*\s*([০-৯]+)'
search_pattern =r'^M-(.*)'
new_file_prefix="PP-"

In [ ]:
# Only for bengali numbers
def bengali_to_int(bengali_numeral):
    bengali_digits = {
        '০': 0, '১': 1, '২': 2, '৩': 3, '৪': 4,
        '৫': 5, '৬': 6, '৭': 7, '৮': 8, '৯': 9
    }
    return int(''.join(str(bengali_digits[digit]) for digit in bengali_numeral))

In [ ]:
def bulk_rename(directory, search_pattern):
    for filename in os.listdir(directory):
        old_path = os.path.join(directory, filename)
        if os.path.isfile(old_path):
            match = re.search(search_pattern, filename)
            if match:
                part_number = match.group(1)
                #only for bengali numbers
                # part_number = bengali_to_int(part_number)
                new_filename = f'{new_file_prefix}{part_number}.mp3'
                new_path = os.path.join(directory, new_filename)
                os.rename(old_path, new_path)
                print(f'Renamed: {filename} -> {new_filename}')

In [ ]:
if __name__ == "__main__":
    directory_path = input(folder_path)
    bulk_rename(directory_path, search_pattern)
    print("Bulk renaming complete!")

In [ ]:
match = re.search(r'^M-(.*)', "M-127.mp3")

In [ ]:
match.group(1)


# Develop cascade from API output

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/Users/joy/akvo/projects/RTMIS/dsl_admin.csv")
df

In [ ]:
df.info()

In [ ]:
# entities = pd.DataFrame(columns=['County', 'Sub-county', 'Ward'])
lst = []

In [ ]:
for index, row in df.iterrows():
    entity_id = row['id']
    parent_id = row['parentid']
    entity_name = row['name']
    entity_level = row['level']

    if entity_level == 4:
        # print(entity_level,entity_name)
        level3_parent = df[df['id'] == parent_id]
        level3_parent_name=level3_parent['name'].values[0]
        level3_parent_parent_id=level3_parent['parentid'].values[0]
        level2=df[df['id'] == level3_parent_parent_id]
        level2_name=level2['name'].values[0]
        lst.append({'County':level2_name, 'Sub-county':level3_parent_name, 'Ward':entity_name})
        # entities=pd.concat([entities,pd.DataFrame({'County':level2_name, 'Sub-county':level3_parent_name, 'Ward':entity_name})])
        

# entities = entities.sort_values(by=['County'])
lst

In [ ]:
len(lst)

In [ ]:
df_ext=pd.DataFrame(lst)
df_ext


In [ ]:

import re
df_ext['Ward'] = df_ext['Ward'].apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
df_ext


In [ ]:
df_ext.info()

In [ ]:
df_ext=df_ext.sort_values(by=['County'],ignore_index=True)
df_ext

In [ ]:
df_ext.to_csv("/Users/joy/akvo/projects/RTMIS/dsl_admin_readable.csv")

# Farmer land size distribution check

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# Suppress all warnings
warnings.filterwarnings("ignore")

#parameters
confidence_level=0.95
p_value_threshold=0.01
variable='cal_actual_income'
variable_title= 'Actual Income'


# Path to the folder containing Excel files
folder_path = "data/SHF/PDC/farmfit_portal_files"

# List to store results
results = []

# Loop through all Excel files in the folder
for file_name in os.listdir(folder_path):
    if not file_name.startswith("~") and file_name.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file_name)
        
        # Load data from the "Cleaned Data" sheet
        df = pd.read_excel(file_path, sheet_name="Cleaned Data")
        # Check if variable column is available in the dataset
        if variable not in df.columns:
            # print(f"Dataset '{file_name}' does not have {variable} column. Skipping.")
            continue
        df = df.dropna(subset=[variable])
        # df = df[df[variable] > 0]  
        # Check if "f_unit_land" is "hectares" and adjust variable accordingly
        # df.loc[df["f_unit_land"] == "hectares", variable] *= 2.47
        # df = df[df[variable] <= 9000]
        # Calculate statistics
        mean_size = df[variable].mean()
        n = len(df[variable])
        confidence_interval = stats.t.interval(confidence_level, n - 1, loc=mean_size, scale=stats.sem(df[variable]))
         # Compute the difference between the upper and lower confidence interval
        confidence_interval_diff = confidence_interval[1] - confidence_interval[0]
        iqr = df[variable].quantile(0.75) - df[variable].quantile(0.25)
        iqr_lower_limit = df[variable].quantile(0.25) - 1.5 * iqr
        iqr_upper_limit = df[variable].quantile(0.75) + 1.5 * iqr
        std_dev = df[variable].std()
        lower_bound = mean_size - 2 * std_dev
        upper_bound = mean_size + 2 * std_dev
        
        # Fit different distributions and find the best fit
        # Expanded list of distributions
        distributions = [
            stats.norm,        # Normal distribution
            stats.expon,       # Exponential distribution
            stats.gamma,       # Gamma distribution
            stats.lognorm,     # Lognormal distribution
            stats.weibull_min, # Weibull distribution
            stats.pareto,      # Pareto distribution
            stats.beta,        # Beta distribution
            stats.exponweib,   # Exponentiated Weibull distribution
            stats.logistic,    # Logistic distribution
            stats.t,           # Student's t-distribution
            # Add more distributions as needed
        ]

        
        best_fit = None
        best_fit_name = ''
        best_fit_params = ()
        best_fit_p_value = np.inf
        
        data = df[variable]
        
        for distribution in distributions:
            params = distribution.fit(data)
            _, p_value = stats.kstest(data, distribution.name, args=params)
            
            if p_value < best_fit_p_value:
                best_fit = distribution
                best_fit_name = distribution.name
                best_fit_params = params
                best_fit_p_value = p_value
                fit_is_good = "Good" if best_fit_p_value > p_value_threshold else "Not Good"
        
        # Append results to the list
        results.append({
            "File Name": file_name,
            f"Mean of {variable_title}": mean_size,
            "Confidence Interval": confidence_interval,
            "CI range" : confidence_interval_diff,
            "IQR": iqr,
            "IQR Lower Limit": iqr_lower_limit,
            "IQR Upper Limit" : iqr_upper_limit,
            "2SD Upper Limit": upper_bound,
            "2SD Lower Limit": lower_bound,
            "Best Fit Distribution": best_fit_name,
            # "Best Fit Params": best_fit_params,
            "Best Fit P Value": best_fit_p_value,
            "Fit Quality": fit_is_good
        })
        plt.figure(figsize=(10, 6))
        # plt.hist(df[variable], bins=20, edgecolor="black")
        # sns.kdeplot(data, shade=True)
         # Overlay the PDF curve of the best fit distribution
        x = np.linspace(min(df[variable]), max(df[variable]), 1000)
        best_fit_distribution = getattr(stats, best_fit_name)
        pdf = best_fit_distribution.pdf(x, *best_fit_params)
        
        # Plot PDF of best fit distribution on a separate y-axis
        fig, ax1 = plt.subplots()
        ax1.set_xlabel(variable_title)
        ax1.set_ylabel("Density (KDE)", color='tab:blue')
        sns.kdeplot(df[variable], shade=True, ax=ax1, color='tab:blue')
        ax1.tick_params(axis='y', labelcolor='tab:blue')

        # ax2 = ax1.twinx()
        # ax2.set_ylabel(f"Probability Density Function (PDF) {best_fit_name}", color='tab:red')
        # sns.kdeplot(df[variable], shade=True, ax=ax1, color='tab:green')
        # # ax2.plot(x, pdf, color='tab:red', label=best_fit_name + " PDF")
        # ax2.tick_params(axis='y', labelcolor='tab:red')


        plt.title(f"{variable_title} Distribution for {file_name}")
        fig.tight_layout()
        plt.show()

# Print the results
results_df = pd.DataFrame(results)
results_df


# Spectogram

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns  # Import seaborn for color mapping



In [ ]:
# Load the audio file
audio_file = "data/audio/nenjinile1-10.wav"
y, sr = librosa.load(audio_file)

In [ ]:

# Generate the spectrogram
D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)

In [ ]:
# Create a frequency axis for the spectrogram
frequencies = librosa.core.fft_frequencies(sr=sr)
dominant_frequencies = frequencies[np.argmax(D, axis=0)]

# Define a mapping of frequencies to notes
def frequency_to_note(frequency):
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    A4_frequency = 440.0  # Frequency of A4 note in Hz
    note_index = round(12 * np.log2(frequency / A4_frequency)) % 12
    octave = round(np.log2(frequency / A4_frequency))
    return f"{note_names[note_index]}{octave}", octave

# Get note labels and octaves for the frequency axis
# note_labels, octaves = zip(*[frequency_to_note(f) for f in frequencies])
note_labels = []
octaves = []

for f in dominant_frequencies:
    try:
        note, octave = frequency_to_note(f)
    except OverflowError:
        # Handle the error here by setting default values
        note = "Unknown"
        octave = 0
    
    note_labels.append(note)
    octaves.append(octave)


In [ ]:
# Create a color map for octaves using seaborn
octave_colors = sns.color_palette("husl", n_colors=len(set(octaves)))

# Display the spectrogram with colored note labels
plt.figure(figsize=(12, 6))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram with Colored Note Labels')

# Add note labels to the frequency axis with colors based on octaves
for i, label in enumerate(note_labels):
    plt.text(-0.5, i, label, fontsize=8, ha="right", va="center", color=octave_colors[octaves[i]])

# Set the color map for the y-axis to match octave colors
plt.yticks(range(0, len(frequencies), 12), ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'])
plt.gca().yaxis.set_tick_params(labelcolor=octave_colors)

plt.show()

In [ ]:
# Compute the spectrogram
D = np.abs(librosa.stft(y))

# Get the number of frames
num_frames = D.shape[1]

# Compute the frame length in seconds
frame_length = len(y) / num_frames / sr

# Create a time axis
times = np.arange(0, num_frames) * frame_length

# Get the frequency bins corresponding to each time frame
frequencies = librosa.core.fft_frequencies(sr=sr)

# Find the frequency bin with the highest magnitude for each time frame
dominant_frequency_bins = []

for frame in np.transpose(D):  # Transpose D to iterate over frames
    if np.all(np.isinf(frame)):
        # Handle the case of all values being infinity
        dominant_frequency_bins.append(0)  # Assign a placeholder value
    else:
        dominant_frequency_bins.append(np.argmax(frame))

# Create lists to store the data for plotting
plot_times = []
plot_frequencies = []
note_labels = []

for i, freq_bin in enumerate(dominant_frequency_bins):
    if freq_bin == 0:
        # Handle the placeholder value for infinite frames
        continue
    
    # Convert frequency bin to Hertz
    dominant_frequency_hz = frequencies[freq_bin]
    
    # Convert Hertz to note name (with error handling)
    try:
        note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        A4_frequency = 440.0  # Frequency of A4 note in Hz
        note_index = round(12 * np.log2(dominant_frequency_hz / A4_frequency)) % 12
        octave = round(np.log2(dominant_frequency_hz / A4_frequency))
        note_label = f"{note_names[note_index]}{octave}"
    except (ValueError, OverflowError, ZeroDivisionError):
        note_label = "N/A"  # Set a placeholder for problematic cases
    
    # Store data for plotting
    plot_times.append(times[i])
    plot_frequencies.append(dominant_frequency_hz)
    note_labels.append(note_label)

# Create a plot showing the dominant frequency at each time frame with note labels
plt.figure(figsize=(12, 6))
plt.plot(plot_times, plot_frequencies, 'bo-', label='Dominant Frequency')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Dominant Frequency Plot from Spectrogram with Note Labels')
plt.grid()

# Add note labels to the plot
for t, f, label in zip(plot_times, plot_frequencies, note_labels):
    plt.text(t, f, label, fontsize=8, ha="right", va="bottom")

plt.legend()
plt.show()

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the audio file
audio_file = "data/audio/nenjinile1-10.wav"
y, sr = librosa.load(audio_file)

# Compute the spectrogram
D = np.abs(librosa.stft(y))

# Get the frequency bins corresponding to each time frame
frequencies = librosa.core.fft_frequencies(sr=sr)

# Find the frequency bin with the highest magnitude for each time frame
dominant_frequency_bins = np.argmax(D, axis=0)

# Convert frequency bins to Hertz
dominant_frequencies = frequencies[dominant_frequency_bins]

# Create a time axis
times = librosa.times_like(dominant_frequencies)

# Convert Hertz to note names and octaves with mapping to the closest note
def frequency_to_note(frequency):
    A4_frequency = 440.0
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    try:
        note_number = 12 * np.log2(frequency / A4_frequency)
        octave = int(note_number) // 12
        note_index = round(int(note_number) % 12)  # Round to the nearest note index
        
        # Handle cases where the calculated note index is outside the 0-11 range
        if note_index < 0:
            note_index = 0
        elif note_index > 11:
            note_index = 11
        
        return f"{note_names[note_index]}{octave}"
    except (OverflowError, ZeroDivisionError):
        return "N/A"  # Set a placeholder for problematic cases

note_labels = [frequency_to_note(f) for f in dominant_frequencies]

# Create a color map for octaves using seaborn
octave_colors = sns.color_palette("husl", n_colors=len(set(note_labels)))

# Display the spectrogram with colored note labels
plt.figure(figsize=(12, 6))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram with Colored Note Labels')

# Add note labels to the frequency axis with colors based on octaves
for i, label in enumerate(note_labels):
    plt.text(-0.5, i, label, fontsize=8, ha="right", va="center", color=octave_colors[i])

# Set the color map for the y-axis to match octave colors
plt.yticks(range(0, len(frequencies), 12), ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'])
plt.gca().yaxis.set_tick_params(labelcolor=octave_colors)

plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the audio file
audio_file = "data/audio/nenjinile1-10.wav"
y, sr = librosa.load(audio_file)

# Compute the spectrogram
D = np.abs(librosa.stft(y))

# Get the frequency bins corresponding to each time frame
frequencies = librosa.core.fft_frequencies(sr=sr)

# Find the frequency bin with the highest magnitude for each time frame
dominant_frequency_bins = np.argmax(D, axis=0)

# Convert frequency bins to Hertz
dominant_frequencies = frequencies[dominant_frequency_bins]

# Create a time axis
times = librosa.times_like(dominant_frequencies)

# Convert Hertz to note names and octaves with mapping to the closest note
def frequency_to_note(frequency):
    A4_frequency = 440.0
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    try:
        note_number = 12 * np.log2(frequency / A4_frequency)
        octave = int(note_number) // 12
        note_index = round(int(note_number) % 12)  # Round to the nearest note index
        
        # Handle cases where the calculated note index is outside the 0-11 range
        if note_index < 0:
            note_index = 0
        elif note_index > 11:
            note_index = 11
        
        return f"{note_names[note_index]}{octave}"
    except (OverflowError, ZeroDivisionError):
        return "N/A"  # Set a placeholder for problematic cases

note_labels = [frequency_to_note(f) for f in dominant_frequencies]

# Create a color map for octaves using seaborn
unique_note_labels = list(set(note_labels))
octave_colors = sns.color_palette("husl", n_colors=len(unique_note_labels))

# Display the spectrogram with colored note labels
plt.figure(figsize=(12, 6))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram with Colored Note Labels')

# Add note labels to the frequency axis with colors based on octaves
for i, label in enumerate(note_labels):
    color_index = unique_note_labels.index(label)  # Find the index of the label in unique_note_labels
    plt.text(-0.5, i, label, fontsize=8, ha="right", va="center", color=octave_colors[color_index])

# Set the color map for the y-axis to match octave colors
plt.yticks(range(0, len(frequencies), 12), ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'])
plt.gca().yaxis.set_tick_params(labelcolor=octave_colors[0:len(frequencies):12])  # Adjust for octave colors

plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the audio file with the correct file path
audio_file = "data/audio/nenjinile1-10.wav"  # Update the file path as needed
y, sr = librosa.load(audio_file)

# Compute the spectrogram
D = np.abs(librosa.stft(y))

# Get the frequency bins corresponding to each time frame
frequencies = librosa.core.fft_frequencies(sr=sr)

# Find the frequency bin with the highest magnitude for each time frame
dominant_frequency_bins = np.argmax(D, axis=0)

# Convert frequency bins to Hertz
dominant_frequencies = frequencies[dominant_frequency_bins]

# Create a time axis
times = librosa.times_like(dominant_frequencies)

# Convert Hertz to note names without octaves
def frequency_to_note(frequency):
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    try:
        note_index = round(12 * np.log2(frequency / 440.0)) % 12  # Calculate note index
        return note_names[note_index]
    except (OverflowError, ZeroDivisionError):
        return "N/A"  # Set a placeholder for problematic cases

note_labels = [frequency_to_note(f) for f in dominant_frequencies]

# Create a color map for note labels using seaborn
unique_note_labels = list(set(note_labels))
note_colors = sns.color_palette("husl", n_colors=len(unique_note_labels))

# Display the spectrogram with colored note labels
plt.figure(figsize=(12, 6))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram with Colored Note Labels')

# Add note labels to the frequency axis with colors
for i, label in enumerate(note_labels):
    color_index = unique_note_labels.index(label)  # Find the index of the label in unique_note_labels
    plt.text(-0.5, i, label, fontsize=8, ha="right", va="center", color=note_colors[color_index])

plt.ylim(80, 600)
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.signal import butter, lfilter

# Load the audio file with the correct file path
audio_file = "data/audio/nenjinile1-10.wav"  # Update the file path as needed
y, sr = librosa.load(audio_file)
# Define the desired frequency range for voice (80 Hz to 2000 Hz)
lowcut = 80.0  # Lower cutoff frequency in Hz
highcut = 2000.0  # Upper cutoff frequency in Hz

# Apply a bandpass filter to the audio signal
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y_filtered = lfilter(b, a, y)
    return y_filtered

# Apply the bandpass filter to the audio
y_f = butter_bandpass(lowcut, highcut, sr)

# Compute the spectrogram
D = np.abs(librosa.stft(y_f))

# Get the frequency bins corresponding to each time frame
frequencies = librosa.core.fft_frequencies(sr=sr)

# Find the frequency bin with the highest magnitude for each time frame
dominant_frequency_bins = np.argmax(D, axis=0)

# Convert frequency bins to Hertz
dominant_frequencies = frequencies[dominant_frequency_bins]

# Create a time axis
times = librosa.times_like(dominant_frequencies)

# Convert Hertz to note names without octaves
def frequency_to_note(frequency):
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    try:
        note_index = round(12 * np.log2(frequency / 440.0)) % 12  # Calculate note index
        return note_names[note_index]
    except (OverflowError, ZeroDivisionError):
        return "N/A"  # Set a placeholder for problematic cases

note_labels = [frequency_to_note(f) for f in dominant_frequencies]

# Create a color map for note labels using seaborn
unique_note_labels = list(set(note_labels))
note_colors = sns.color_palette("husl", n_colors=len(unique_note_labels))

# Display the spectrogram with colored note labels
plt.figure(figsize=(12, 6))
librosa.display.specshow(
    librosa.amplitude_to_db(D, ref=np.max), sr=sr, x_axis='time', y_axis='hz', cmap='viridis'
)
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram with Colored Note Labels')

# Add note labels to the frequency axis with colors
for i, label in enumerate(note_labels):
    color_index = unique_note_labels.index(label)  # Find the index of the label in unique_note_labels
    plt.text(-0.5, i, label, fontsize=8, ha="right", va="center", color=note_colors[color_index])

plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file using librosa
audio_file = "data/audio/nenjinile1-10.wav"  # Update the file path as needed
y, sr = librosa.load(audio_file, duration=40.0)  # Load the first 40 seconds

# Spectrogram using 'spectrogram' command
plt.figure(figsize=(8, 4))
plt.subplot(2, 1, 1)
S = librosa.stft(y, n_fft=5000, hop_length=240, win_length=5000)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sr, x_axis='time', y_axis='hz')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram using spectrogram')

# Spectrogram using short-time Fourier transform 'stft'
plt.subplot(2, 1, 2)
wlen = 5000  # Window length
h = 400  # Overlap is wlen - h
S = librosa.stft(y, n_fft=wlen, hop_length=h, win_length=wlen, center=False)
librosa.display.specshow(np.log10(np.abs(S)), sr=sr, x_axis='time', y_axis='hz')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram using stft')

plt.tight_layout()
plt.show()



In [ ]:
import librosa
import scipy.signal as signal
import soundfile as sf

# Load the audio file using librosa
audio_file = "data/audio/nenjinile1-10.wav"  # Update the file path as needed
y, sr = librosa.load(audio_file, duration=40.0, mono=False)  # Load the first 40 seconds in stereo

# Define the frequency range for the bandpass filter (200 Hz to 600 Hz)
lowcut = 200  # Low cutoff frequency in Hz
highcut = 600  # High cutoff frequency in Hz

# Design a bandpass filter
nyquist = 0.5 * sr
low = lowcut / nyquist
high = highcut / nyquist
b, a = signal.butter(6, [low, high], btype='band')

# Apply the bandpass filter to each channel of the stereo audio
filtered_left = signal.lfilter(b, a, y[0, :])
filtered_right = signal.lfilter(b, a, y[1, :])

# Combine the filtered left and right channels into a stereo signal
filtered_audio = np.vstack((filtered_left, filtered_right))

# Save the filtered stereo audio to a new file
filtered_audio_file = "data/audio/FS_nenjinile1-10.wav"  # Update the file name as needed
sf.write(filtered_audio_file, filtered_audio.T, sr)

print(f"Filtered stereo audio saved to {filtered_audio_file}")

